In [1]:
import pandas as pd
import os

In [15]:
RAW_DATA_DIR = "/home/joao/Documents/ONS_CED_TimeSeries/raw_data"
PROCESSED_DATA_DIR = '/home/joao/Documents/ONS_CED_TimeSeries/processed_data'

all_dfs = []

In [ ]:
print(f"Reading parquet files from: {RAW_DATA_DIR}")

for year in range(2000, 2026): 
    file_name = f"CARGA_ENERGIA_{year}.parquet"
    file_path = os.path.join(RAW_DATA_DIR, file_name)

    if os.path.exists(file_path):
        print(f"Lendo {file_name}...")
        try:
            df_year = pd.read_parquet(file_path)
            all_dfs.append(df_year)
        except Exception as e:
            print(f"Erro reading file {file_name}: {e}")
    else:
        print(f"File not found: {file_name}. Skipping file")

Iniciando a leitura dos arquivos Parquet de 2000 a 2005 do diretório: /home/joao/Documents/ONS_CED_TimeSeries/raw_data
Lendo CARGA_ENERGIA_2000.parquet...
Lendo CARGA_ENERGIA_2001.parquet...
Lendo CARGA_ENERGIA_2002.parquet...
Lendo CARGA_ENERGIA_2003.parquet...
Lendo CARGA_ENERGIA_2004.parquet...
Lendo CARGA_ENERGIA_2005.parquet...
Lendo CARGA_ENERGIA_2006.parquet...
Lendo CARGA_ENERGIA_2007.parquet...
Lendo CARGA_ENERGIA_2008.parquet...
Lendo CARGA_ENERGIA_2009.parquet...
Lendo CARGA_ENERGIA_2010.parquet...
Lendo CARGA_ENERGIA_2011.parquet...
Lendo CARGA_ENERGIA_2012.parquet...
Lendo CARGA_ENERGIA_2013.parquet...
Lendo CARGA_ENERGIA_2014.parquet...
Lendo CARGA_ENERGIA_2015.parquet...
Lendo CARGA_ENERGIA_2016.parquet...
Lendo CARGA_ENERGIA_2017.parquet...
Lendo CARGA_ENERGIA_2018.parquet...
Lendo CARGA_ENERGIA_2019.parquet...
Lendo CARGA_ENERGIA_2020.parquet...
Lendo CARGA_ENERGIA_2021.parquet...
Lendo CARGA_ENERGIA_2022.parquet...
Lendo CARGA_ENERGIA_2023.parquet...
Lendo CARGA_ENERG

In [ ]:
# Concat dataframes
if all_dfs:
    df_combined = pd.concat(all_dfs, ignore_index=True)
    print(f"\nAll files were combined. Total lines: {len(df_combined)}")
    print("First 5 rows before processing")
    print(df_combined.head())
    print("\nCombined dataframe info")
    df_combined.info()
else:
    print("\nDidnt read any file")
    exit() 


Todos os arquivos foram combinados. Total de linhas: 37165
Primeiras 5 linhas do DataFrame combinado (antes do processamento):
  id_subsistema        nom_subsistema din_instante val_cargaenergiamwmed
0             N                 Norte   2000-01-01         2243.51250000
1            NE              Nordeste   2000-01-01         4646.37083333
2             S                   Sul   2000-01-01         4800.65000000
3            SE  Sudeste/Centro-Oeste   2000-01-01        19045.99583333
4             N                 Norte   2000-01-02         2259.80833333

Informações sobre o DataFrame combinado (antes do processamento):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37165 entries, 0 to 37164
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id_subsistema          37165 non-null  object        
 1   nom_subsistema         37165 non-null  object        
 2   din_instante      

In [ ]:
print("\nConverting 'din_instante' to datetime...")
df_combined['din_instante'] = pd.to_datetime(df_combined['din_instante'])


Convertendo 'din_instante' para o tipo datetime...


In [ ]:
print("Converting 'val_cargaenergiamwmed' to (float)...")
df_combined['val_cargaenergiamwmed'] = pd.to_numeric(df_combined['val_cargaenergiamwmed'], errors='coerce')

Convertendo 'val_cargaenergiamwmed' para tipo numérico (float)...


In [ ]:
print("Grouping by 'din_instante' e sum 'val_cargaenergiamwmed'...")
df_daily_total = df_combined.groupby('din_instante')['val_cargaenergiamwmed'].sum().reset_index()

Agrupando os dados por 'din_instante' e somando 'val_cargaenergiamwmed'...


In [8]:
df_daily_total = df_daily_total.rename(columns={'val_cargaenergiamwmed': 'total_carga_mwmed'})

In [ ]:
print("First 5 rows:")
print(df_daily_total.head())


Processamento concluído.
Primeiras 5 linhas do DataFrame final com a carga total diária:
  din_instante  total_carga_mwmed
0   2000-01-01       30736.529167
1   2000-01-02       31453.354167
2   2000-01-03       37466.737500
3   2000-01-04       39149.970833
4   2000-01-05       39961.879167


In [ ]:
print("\nLast 5 rows:")
print(df_daily_total.tail())


Últimas 5 linhas do DataFrame final com a carga total diária:
     din_instante  total_carga_mwmed
9287   2025-06-04       79512.124167
9288   2025-06-05       79860.350042
9289   2025-06-06           0.000000
9290   2025-06-07           0.000000
9291   2025-06-08           0.000000


In [ ]:
print("\nRemoving last 3 rows that contains 0")
df_daily_total = df_daily_total.iloc[:-3]


Removendo as últimas 3 linhas do DataFrame...


In [ ]:
print(df_daily_total.head())
print(df_daily_total.tail())


Primeiras 5 linhas do DataFrame após a remoção:
  din_instante  total_carga_mwmed
0   2000-01-01       30736.529167
1   2000-01-02       31453.354167
2   2000-01-03       37466.737500
3   2000-01-04       39149.970833
4   2000-01-05       39961.879167

Últimas 5 linhas do DataFrame após a remoção:
     din_instante  total_carga_mwmed
9284   2025-06-01       65715.325542
9285   2025-06-02       76922.647583
9286   2025-06-03       79365.290958
9287   2025-06-04       79512.124167
9288   2025-06-05       79860.350042


In [ ]:
df_daily_total.info()


Informações sobre o DataFrame final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9289 entries, 0 to 9288
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   din_instante       9289 non-null   datetime64[ns]
 1   total_carga_mwmed  9289 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 145.3 KB


In [16]:
save_path = os.path.join(PROCESSED_DATA_DIR, "carga_energia_diaria_total_2000_2025.parquet")
df_daily_total.to_parquet(save_path, index=False)
print(f"\nDataFrame processado salvo em: {save_path}")


DataFrame processado salvo em: /home/joao/Documents/ONS_CED_TimeSeries/processed_data/carga_energia_diaria_total_2000_2025.parquet
